In [ ]:
from processador_ibovespa import ProcessadorIbovespa

caminho_arquivo = "../../dados/investimento/ibovespa.csv"
processador = ProcessadorIbovespa(caminho_arquivo)
features_desejadas = ['dias_semana', 'lags', 'diffs', 'volatilidade', 'medias_moveis', 'nulos_tratados']

df_final = processador.get_dataframe(features=features_desejadas)
df_final.head()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_final.drop(['Data'], axis=1))

X = df_scaled[:, 1:]  
y = df_scaled[:, 0]   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=100, batch_size=32)


In [ ]:
# Avaliando o modelo
loss = model.evaluate(X_test, y_test)
print(f"Loss no Teste: {loss}")

# Previsões
y_pred = model.predict(X_test)

# Redimensionar y_pred para garantir a compatibilidade de forma
y_pred = y_pred.reshape(-1)

# Preparando dados para reverter a normalização (assumindo que 'Último' é a primeira coluna no scaler)
y_pred_expanded = np.concatenate([y_pred.reshape(-1, 1), np.zeros((y_pred.shape[0], df_scaled.shape[1] - 1))], axis=1)
y_test_expanded = np.concatenate([y_test.reshape(-1, 1), np.zeros((y_test.shape[0], df_scaled.shape[1] - 1))], axis=1)

# Reverter a normalização para interpretar as previsões no espaço original
y_pred_original = scaler.inverse_transform(y_pred_expanded)[:, 0]
y_test_original = scaler.inverse_transform(y_test_expanded)[:, 0]


In [ ]:
import matplotlib.pyplot as plt

# Plotando valores reais vs previsões
plt.figure(figsize=(12, 6))
plt.plot(y_test_original, label='Valores Reais')
plt.plot(y_pred_original, label='Previsões', alpha=0.7)
plt.title('Valores Reais vs Previsões')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

# Calculando MAPE
mape = mean_absolute_percentage_error(y_test_original, y_pred_original)
print(f"MAPE: {mape}")

# Calculando WMAPE
wmape = np.sum(np.abs(y_test_original - y_pred_original)) / np.sum(y_test_original)
print(f"WMAPE: {wmape}")


In [ ]:
residuos = y_test_original - y_pred_original

plt.figure(figsize=(12, 6))
plt.plot(residuos)
plt.title('Resíduos das Previsões')
plt.xlabel('Tempo')
plt.ylabel('Resíduo')
plt.show()

In [ ]:
limite_aceitavel = 0.3  # 5% de erro
acertos = np.sum(np.abs((y_pred_original - y_test_original) / y_test_original) < limite_aceitavel)
acuracia = acertos / len(y_test_original)

print(f"Acurácia (com base em um limite de erro de {limite_aceitavel * 100}%): {acuracia * 100}%")


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Função para calcular a importância de permutação
def permutation_importance(model, X, y, metric):
    baseline_score = metric(y, model.predict(X))
    importance = {}
    for i in range(X.shape[2]):  # X.shape[2] é o número de features
        X_permuted = X.copy()
        np.random.shuffle(X_permuted[:, :, i])  # Embaralhar a feature i
        permuted_score = metric(y, model.predict(X_permuted))
        importance[i] = baseline_score - permuted_score  # Diferença no desempenho
    return importance

# Redimensionando X_test para o formato do LSTM se necessário
X_test_transformed = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Calculando a importância
importance_scores = permutation_importance(wrapped_model, X_test_transformed, y_test, mean_squared_error)

# Imprimindo a importância
for i, score in importance_scores.items():
    print(f"Feature {i} - Score: {score}")
